In [4]:
def run_average(img, accumWt):
    global background
    # initialize the background
    if background is None:
        background = img.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(img, background, accumWt)


In [5]:
def count_fingers(thresholded, segmented):
    # find the convex hull of the segmented hand region
    con_hull = cv2.convexHull(segmented)

    # find the most extreme points in the convex hull
    extreme_top    = tuple(con_hull[con_hull[:, :, 1].argmin()][0])
    extreme_bottom = tuple(con_hull[con_hull[:, :, 1].argmax()][0])
    extreme_left   = tuple(con_hull[con_hull[:, :, 0].argmin()][0])
    extreme_right  = tuple(con_hull[con_hull[:, :, 0].argmax()][0])

    # find the center of the palm
    cX = int((extreme_left[0] + extreme_right[0]) / 2)
    cY = int((extreme_top[1] + extreme_bottom[1]) / 2)

    # find the maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[extreme_left, extreme_right, extreme_top, extreme_bottom])[0]
    maximum_distance = distance[distance.argmax()]

    # calculate the radius of the circle with 80% of the max euclidean distance obtained
    radius = int(0.8 * maximum_distance)

    # find the circumference of the circle
    circumference = (2 * np.pi * radius)

    # take out the circular region of interest which has 
    # the palm and the fingers
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
	
    # draw the circular ROI
    cv2.circle(circular_roi, (cX, cY), radius, 255, 1)

    # take bit-wise AND between thresholded hand using the circular ROI as the mask
    # which gives the cuts obtained using mask on the thresholded hand image
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    # compute the contours in the circular ROI
    ( cont, _) = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # initalize the finger count
    count = 0

    # loop through the contours found
    for c in cont:
        # compute the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)

        # increment the count of fingers only if -
        # 1. The contour region is not the wrist (bottom area)
        # 2. The number of points along the contour does not exceed
        #     25% of the circumference of the circular ROI
        if ((cY + (cY * 0.25)) > (y + h)) and ((circumference * 0.25) > c.shape[0]):
            count += 1

    return count


In [ ]:
def segment(img, threshold=25):
    global background

    diff = cv2.absdiff(background.astype("uint8"), img)


    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # detecting the contours in the thresholded image
    (cont, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected i.e no fingers present
    if len(cont) == 0:
        return
    else:
        # based on contour area, getting the maximum contour which most of the times is the hand
        segmented = max(cont, key=cv2.contourArea)
        return (thresholded, segmented)


In [9]:

#importing all the necessary libraries


import cv2
import imutils
import numpy as np
from sklearn.metrics import pairwise

# global variable

background = None

number_list = []

text = np.ones((1000, 1000, 3), np.uint8)
text = text * 255

if __name__ == "__main__":

    accumWeight = 0.5

    # creating the video capture object
    cap = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590


    num_frames = 0


    calibrated = False


    while(True):
        # getting the current frame
        (grabbed, frame) = cap.read()

        # resizing the frame
        frame = imutils.resize(frame, width=700)

        # flipping the frame horizontal axis to avoid the mirror view
        frame = cv2.flip(frame, 1)

        # copying the frame
        clone = frame.copy()

        # getting the height and width of the frame
        (height, width) = frame.shape[:2]

        # getting the ROI
        roi = frame[top:bottom, right:left]

        # converting the roi to grayscale and blur it for easy image processing and noise removal
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5,5), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our weighted average model gets calibrated
        if num_frames < 30:
            run_average(gray, accumWeight)
            if num_frames == 1:
                print("Please wait! calibrating...")
            elif num_frames == 29:
                print("Calibration successfull...")
        else:
            # segmenting the hand region
            hand = segment(gray)

            # checking whether hand region is segmented
        if hand is not None:

                (thresholded, segmented) = hand

                # drawing the segmented region and displaying the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                # counting the number of fingers
                fingers = count_fingers(thresholded, segmented)
            
                number_list.append(fingers)

                cv2.putText(clone, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
#                 cv2.putText(clone, str(number_list), (0, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
                cv2.putText(text, str(number_list), (0, 0), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
                # diplaying the thresholding 
                cv2 .imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # displaying frames
        cv2.imshow("Main", clone)
        cv2.imshow('Text',text)

        k = cv2.waitKey(1) & 0xFF

        # Press Esc for the program to end
        if k == 27:
            break


cap.release()
cv2.destroyAllWindows()

Please wait! calibrating...
Calibration successfull...
